# Simulated Vibronic Spectrum

In [1]:
%aiida
import numpy as np
import math
from matplotlib import pyplot
from scipy import constants
import ipywidgets as ipw
from IPython.display import clear_output

In [2]:
wc = load_node(3458)

In [3]:
charged_out = wc.outputs.charged_out.get_dict()
neutral_out = wc.outputs.neutral_out.get_dict()

In [4]:
charged_coordinates = np.array(charged_out['atomcoords'][-1])
neutral_coordinates = np.array(neutral_out['atomcoords'][-1])
atom_displacement = (charged_coordinates - neutral_coordinates)
masses = neutral_out['atommasses']

In [5]:
# I get error in 6th digit wrt Nil's results
# it looks like the aiida parser is not precise enough.
mass_weighted_displacement = (atom_displacement.transpose() * np.sqrt(masses)).transpose()

In [6]:
displacements = np.array(charged_out['vibdisps'])

In [7]:
nominator = (displacements.transpose([0, 2,1]) * np.sqrt(masses)).transpose([0, 2, 1])
#denominator = np.sum(np.einsum('ijk,ijk->ij', nominator, nominator), axis=1)
# the one below is the same, but simpler
denominator = np.sqrt(np.sum(nominator**2, axis=(1,2)))
normal_modes = (nominator.transpose([2,1, 0])/denominator).transpose([2,1,0])

In [8]:
vibrational_mode_amplitudes = np.einsum("ijk,jk->i", normal_modes, mass_weighted_displacement)

In [9]:
# 0.0001239842573148 eV = 1 cm^-1
vibrational_energies = np.array(charged_out['vibfreqs'])

In [10]:
huang_rhys_factors = np.abs(vibrational_energies) * vibrational_mode_amplitudes**2 \
    * np.pi * 2.99792458 / 1.054571817 * 1.66053906660 * 1e-3

In [11]:
vibrational_energies *= 0.0001239842573148

In [12]:
def put_intensities_on_energy_grid(intensities, vibrational_energy, energy_resolution):
    number_of_harmonics = len(intensities)
    grid_size = round(vibrational_energy * (number_of_harmonics -1) / energy_resolution) + 1
    grid = np.zeros(grid_size)
    for i, intensity in enumerate(intensities):
        n_point = round(vibrational_energy * i / energy_resolution)
        grid[n_point] = intensity
    return grid

def lorentzian(x, g):
    return g / np.pi * 0.5 / (x**2 + 0.25 * g**2)

def lorentzian_borders(g, t, energy_resolution):
    value = g / 2.0 * np.sqrt((1.0 - t)/t)
    return np.linspace(-value, value, int(np.round(value*2/energy_resolution)))

In [13]:
out = ipw.Output()
def changed(_=None):
    global intensities, ife, ife_with_broadering
    with out:

        intensities = []
        for hrf in huang_rhys_factors:
            intensities.append([])
            intensity = max_intensity = np.exp(-hrf)
            n=1
            while intensity/max_intensity > intensity_threshold_widget.value:
                intensities[-1].append(intensity)
                intensity = np.exp(-hrf) * (hrf**n) / math.factorial(n)
                max_intensity = max(intensity, max_intensity)
                n+=1

        ife = [1]
        for i, intensity in enumerate(intensities):
            intens_on_grid = put_intensities_on_energy_grid(intensity, vibrational_energies[i], er_widget.value)
            ife = np.convolve(intens_on_grid, ife)
        
        x_points_lorentzian = lorentzian_borders(fwhm_widget.value, intensity_threshold_widget.value, er_widget.value)
        lp = lorentzian(
            x_points_lorentzian,
            fwhm_widget.value,
        )
        ife_with_broadering = np.convolve(ife, lp)
        ife_with_broadering /= np.max(ife_with_broadering)
        #ife_with_broadering = np.convolve(ife, lp)
        clear_output()
        x_axis = np.linspace(0, er_widget.value*len(ife_with_broadering), len(ife_with_broadering)) + x_points_lorentzian[0]
        x_axis *= 1000
        #print(ife_with_broadering.shape)
        pyplot.plot(x_axis, ife_with_broadering)
        pyplot.xlabel("Energy (meV)")
        pyplot.xlim([min_x.value, max_x.value])
        pyplot.show()

In [14]:
er_widget = ipw.FloatLogSlider(
    value=0.1,
    base=10,
    min=-5, # max exponent of base
    max=-1, # min exponent of base
    step=0.2, # exponent step
    description='Energy resolution',
    style={'description_width': 'initial'},
)
er_widget.observe(changed, names='value')

fwhm_widget = ipw.FloatLogSlider(
    value=0.006,
    base=10,
    min=-4, # max exponent of base
    max=0, # min exponent of base
    step=0.2, # exponent step
    description='FWHM ',
    style={'description_width': 'initial'},
)
fwhm_widget.observe(changed, names='value')

intensity_threshold_widget = ipw.FloatLogSlider(
    value=0.02,
    base=10,
    min=-4, # max exponent of base
    max=0, # min exponent of base
    step=0.2, # exponent step
    description='Intensity threshold ',
    style={'description_width': 'initial'},
)

intensity_threshold_widget.observe(changed, names='value')

min_x = ipw.FloatText(
    value=0,
    description='Min X:',
    disabled=False
)

max_x = ipw.FloatText(
    value=700,
    description='Max X:',
    disabled=False
)

In [15]:
display(er_widget, fwhm_widget, intensity_threshold_widget, ipw.HBox([min_x, max_x]), out)

FloatLogSlider(value=0.1, description='Energy resolution', max=-1.0, min=-5.0, step=0.2, style=SliderStyle(des…

FloatLogSlider(value=0.006, description='FWHM ', max=0.0, min=-4.0, step=0.2, style=SliderStyle(description_wi…

FloatLogSlider(value=0.02, description='Intensity threshold ', max=0.0, min=-4.0, step=0.2, style=SliderStyle(…

Output()